In [2]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score,roc_curve

In [3]:
loaded_model = xgb.Booster()
# loaded_model.load_model('BBILA_60_in_15mob.json')
loaded_model.load_model('LeoPayu_BBILA_60_in_15mob_Vin.json')
print("Model Loaded Successfully")

Model Loaded Successfully


In [20]:
#load model developing data
model_data = pd.read_parquet('Leobrix_Release_BBILAP_Combined_Data.parquet.gzip')
print('********* Data loaded ********* ')
print(model_data.shape)
# Make sure no duplicates in the data
model_data =  model_data.drop_duplicates(subset=['unique_reference_number'])
model_data =  model_data.drop_duplicates(subset=['latest_feature_id'])
print(model_data.shape)


********* Data loaded ********* 
(478328, 2831)
(478328, 2831)


In [21]:
def map_weight(value):
    if (value >= '2022-01') and (value <= '2022-06'):
        return 2
    elif (value >= '2021-07') and (value <= '2021-12'):
        return 1
    elif (value >= '2021-01') and (value <= '2021-06'):
        return 0.5
    else:
        return 1

model_data['Weight_vintage'] = model_data['month_year'].apply(map_weight)


In [26]:
# Drop Null values of Target 
df = model_data.dropna(subset=['90_in_24mob'])
# take one copy for model development data
data = df.copy()
#some bank data have inf very rate but by safe replace 
data.replace([np.inf, -np.inf], np.nan, inplace=True)

In [27]:
# check unqiue latest feature id
model_data['latest_feature_id'].nunique()

478328

In [28]:
model_data['unique_reference_number'].nunique()

478327

In [29]:
model_data =  model_data.drop_duplicates(subset=['unique_reference_number'])

In [30]:
model_data['90_in_24mob'].value_counts(dropna=False)

90_in_24mob
NaN    424397
0.0     44727
1.0      9204
Name: count, dtype: int64

In [31]:
df = model_data.dropna(subset=['90_in_24mob'])

In [32]:
df['90_in_24mob'].value_counts(dropna=False)

90_in_24mob
0.0    44727
1.0     9204
Name: count, dtype: int64

In [33]:
df['user_segment'].value_counts()

user_segment
OFFUS    38474
ONUS     15457
Name: count, dtype: int64

In [34]:
# take one copy for model development data
data = df.copy()
data.replace([np.inf, -np.inf], np.nan, inplace=True)

In [35]:
loaded_model.feature_names

['num_PERSONAL LOAN_enq_last_6M',
 'num_CONSUMER LOAN_enq_last_6M_36M',
 'Num_of_PERSONAL LOAN_opened_last6M_12M',
 'num_CREDIT CARD_enq_last_12M',
 'Utilization_all',
 'Total_outstanding_Balance_CREDIT CARD_opened_last_9M',
 'highest_sanction_amount_credit_cards',
 'Age_of_oldest_open_account',
 'Utilization_CREDIT CARD_opened_last_36M',
 'total_emi',
 'total_credit_all_open',
 'bounced_transactions_count',
 'Min_Balance_Min_M1_M2_M3',
 'num_credit_gt_salary',
 'upi_trans_count',
 'Max_Credit_Avg_M1_M2_M3',
 'ratio_total_debit_to_total_credit_M3',
 'ratio_balance_after_2_days_max_credit_M3',
 'ratio_5th_day_balance_to_max_credit_M3',
 'deviation_from_median_income/standard_deviation_income#last_365_Days',
 'loan_amount_to_income#last_365_Days',
 'aggs907',
 'dm206s',
 'g406s',
 'percentage_ever_delinquent',
 'months_since_most_recent_delinquency_installment',
 'months_since_most_recent_delinquency_all',
 'percentage_utilization_gt_75',
 'No of enq in past 90days',
 'Num_Enq_non_BANK_l

In [36]:
model_df1 = data[(data['month_year']>='2021-01') & (data['month_year']<='2022-06')]
oot_df1= data[(data['month_year']>='2022-07') & (data['month_year']<='2022-09')]

In [38]:
features_list = [
'num_PERSONAL LOAN_enq_last_6M',
 'num_CONSUMER LOAN_enq_last_6M_36M',
 'Num_of_PERSONAL LOAN_opened_last6M_12M',
 'num_CREDIT CARD_enq_last_12M',
 'Utilization_all',
 'Total_outstanding_Balance_CREDIT CARD_opened_last_9M',
 'highest_sanction_amount_credit_cards',
 'Age_of_oldest_open_account',
 'Utilization_CREDIT CARD_opened_last_36M',
 'total_emi',
 'total_credit_all_open',
 'bounced_transactions_count',
 'Min_Balance_Min_M1_M2_M3',
 'num_credit_gt_salary',
 'upi_trans_count',
 'Max_Credit_Avg_M1_M2_M3',
 'ratio_total_debit_to_total_credit_M3',
 'ratio_balance_after_2_days_max_credit_M3',
 'ratio_5th_day_balance_to_max_credit_M3',
 'deviation_from_median_income/standard_deviation_income#last_365_Days',
 'loan_amount_to_income#last_365_Days',
 'aggs907',
 'dm206s',
 'g406s',
 'percentage_ever_delinquent',
 'months_since_most_recent_delinquency_installment',
 'months_since_most_recent_delinquency_all',
 'percentage_utilization_gt_75',
 'No of enq in past 90days',
 'Num_Enq_non_BANK_last_12M',
 'Num_of_installment_loan_opened',
'month_year',
'90_in_24mob']

In [39]:
# Selected Features for model build purpose
model_df = model_df1[features_list]

In [40]:
oot_df = oot_df1[features_list]

In [41]:
model_df.shape, oot_df.shape

((39010, 33), (12128, 33))

In [42]:
model_df['90_in_24mob'].value_counts(dropna=False)

90_in_24mob
0.0    32389
1.0     6621
Name: count, dtype: int64

In [43]:
# Separate features (X) and target (y) for Model and OOT datasets
X_model = model_df.drop(columns=['month_year', '90_in_24mob']) # Adjust 'target' to your actual target column name
y_model = model_df['90_in_24mob']

In [44]:
X_oot = oot_df.drop(columns=['month_year', '90_in_24mob'])
y_oot = oot_df['90_in_24mob']

In [45]:
#Handling Null Values
for i in X_model.columns:
    X_model[i].fillna(-9999,inplace=True)

/tmp/ipykernel_3180444/2233446063.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  X_model[i].fillna(-9999,inplace=True)


In [46]:
for i in X_oot.columns:
    X_oot[i].fillna(-9999,inplace=True)

/tmp/ipykernel_3180444/1979277635.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  X_oot[i].fillna(-9999,inplace=True)


In [47]:
X_oot.shape,y_oot.shape

((12128, 31), (12128,))

In [48]:
X_train, X_test, y_train, y_test = train_test_split(X_model, y_model, test_size=0.40,random_state=42)

In [49]:
X_train.shape, X_test.shape

((23406, 31), (15604, 31))

In [50]:
# 2. **Prepare DMatrix for XGBoost**
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)
doot = xgb.DMatrix(X_oot, label=y_oot)


In [51]:
def decile_analysis(df, prediction_col, actual_col,label_good= 0, label_bad= 1 ):
    
    df['Decile'] = pd.qcut(df[prediction_col],10, duplicates='drop', labels = False)+1
    # count_good = df.groupby("Decile")[actual_col]==0.shape[0]
    # count_bad = df.groupby("Decile")[actual_col]==1.shape[0]
   
    decile_summary = df.groupby('Decile').agg(
        mean_actual = (actual_col, 'mean'),
        mean_prediction =( prediction_col, 'mean'),
        max_prediction =( prediction_col, 'max'),
        min_prediction =( prediction_col, 'min'),
        
        count=(actual_col, 'count'),
        label_good=(actual_col,lambda x: (x == label_good).sum()),
        label_bad=(actual_col,lambda x: (x == label_bad).sum())
    ).sort_index(ascending =False)
        
        # label_good =(actual_col == label_good).sum(),
        # label_bad =(actual_col == label_good).sum()).sort_index(ascending =False)
        
    auc = roc_auc_score(df[actual_col], df[prediction_col])
    gini = 2*auc-1
    fpr,tpr,thresholds = roc_curve(df[actual_col], df[prediction_col])
    ks = (tpr-fpr).max()
    return decile_summary, auc, gini, ks
    #print(f'Gini:{gini}, AUC:{auc}, KS:{ks}')

In [52]:
# params = {
#     'objective': 'binary:logistic',
#     'eval_metric': 'auc',
#     'learning_rate': 0.01,
#      'max_depth': 2,

#     'subsample': 0.3,
#     'colsample_bytree': 0.3,
#     # 'min_child_weight':25,

#     #'lambda': 2.6,
#     'lambda': 7.5,
#     'alpha':5.5,
#     'gamma':5.0,

#     # 'scale_pos_weight':4.0,
#     'random_state': 24
    
# }

# evals = [(dtrain, 'train'), (dtest, 'eval')]
# xgb_model = xgb.train(
#     params,
#     dtrain,
#     num_boost_round=500,
#     early_stopping_rounds=50,
#     evals=evals,
#     verbose_eval=10
# )

In [53]:
# 4. **Predictions**
X_train['y_actual'] = y_train
X_train['60_15_y_pred'] = loaded_model.predict(dtrain)
X_train['LID'] = model_df1.loc[X_train.index,'latest_feature_id'].values

X_test['y_actual'] = y_test
X_test['60_15_y_pred'] = loaded_model.predict(dtest)
X_test['LID'] = model_df1.loc[X_test.index,'latest_feature_id'].values

X_oot['y_actual'] = y_oot
X_oot['60_15_y_pred'] = loaded_model.predict(doot)
X_oot['LID'] = oot_df1.loc[X_oot.index,'latest_feature_id'].values

In [54]:
# 6. **Run Decile Analysis**
train_summary, train_auc, train_gini, train_ks = decile_analysis(X_train, '60_15_y_pred', 'y_actual')
test_summary, test_auc, test_gini, test_ks = decile_analysis(X_test, '60_15_y_pred', 'y_actual')
oot_summary, oot_auc, oot_gini, oot_ks = decile_analysis(X_oot, '60_15_y_pred', 'y_actual')

# 7. **Print Results**
print(f"Train AUC: {train_auc:.2f}, Gini: {train_gini:.2f}, KS: {train_ks:.2f}")
print(f"Test AUC: {test_auc:.2f}, Gini: {test_gini:.2f}, KS: {test_ks:.2f}")
print(f"OOT AUC: {oot_auc:.2f}, Gini: {oot_gini:.2f}, KS: {oot_ks:.2f}")


Train AUC: 0.67, Gini: 0.35, KS: 0.25
Test AUC: 0.68, Gini: 0.35, KS: 0.26
OOT AUC: 0.68, Gini: 0.37, KS: 0.27


In [55]:
Train = X_train[['LID','y_actual','60_15_y_pred']]
Test = X_test[['LID','y_actual','60_15_y_pred']]
Oot = X_oot[['LID','y_actual','60_15_y_pred']]


In [56]:
Train.to_csv('BBILA_Train_Pred_60_15_Vin.csv',index=False)
Test.to_csv('BBILA_Test_Pred_60_15_Vin.csv',index=False)
Oot.to_csv('BBILA_Oot_Pred_60_15_Vin.csv',index=False)


In [81]:
loaded_model.feature_names

['num_PERSONAL LOAN_enq_last_24M',
 'num_CONSUMER LOAN_enq_last_9M',
 'total_emi',
 'num_of_loans_le_1lac_non_credit',
 'num_CREDIT CARD_enq_last_3M',
 'Utilization_all',
 'Total_outstanding_Balance_CREDIT CARD_opened_last_6M',
 'Age_of_oldest_open_account',
 'Utilization_CREDIT CARD_opened_last_36M',
 'highest_sanction_amount_non_credit',
 'Total_high_credit',
 'num_PL_LE_1_Lac_opened_in_last_3M',
 'cash_withdrawal_sum',
 'balance_5th_day_M2',
 'bounced_transactions_count',
 'Min_Balance_Min_M1_M2_M3',
 'num_credit_gt_salary',
 'upi_trans_count',
 'Max_Credit_Avg_M1_M2_M3',
 'ratio_total_debit_to_total_credit_M3',
 'ratio_6th_day_balance_to_2th_day_balance_M1',
 'percentage_ever_delinquent',
 'months_since_most_recent_delinquency_all',
 'percentage_utilization_gt_75',
 'No of enq in past 90days',
 'Num_Enq_non_BANK_last_12M',
 'Num_of_installment_loan_opened',
 'g406s',
 'dm206s',
 'statedIncome',
 'deviation_from_mean_income#last_30_Days',
 'deviation_from_median_income/standard_devi

In [58]:
loaded_model = xgb.Booster()
loaded_model.load_model('LeoPayu_BBILA_15_in_24mob_Vin.json')
print("Model Loaded Successfully")

Model Loaded Successfully


In [59]:
#load model developing data
model_data = pd.read_parquet('Leobrix_Release_BBILAP_Combined_Data.parquet.gzip')
print('********* Data loaded ********* ')
print(model_data.shape)
# Make sure no duplicates in the data
model_data =  model_data.drop_duplicates(subset=['unique_reference_number'])
model_data =  model_data.drop_duplicates(subset=['latest_feature_id'])
print(model_data.shape)

********* Data loaded ********* 
(478328, 2831)
(478328, 2831)


In [60]:
# Drop Null values of Target 
df = model_data.dropna(subset=['90_in_24mob'])
# take one copy for model development data
data = df.copy()
#some bank data have inf very rate but by safe replace 
data.replace([np.inf, -np.inf], np.nan, inplace=True)

In [61]:

model_df1 = data[(data['month_year']>='2021-01') & (data['month_year']<='2022-06')]
oot_df1= data[(data['month_year']>='2022-07') & (data['month_year']<='2022-09')]


In [62]:
loaded_model.feature_names

['num_PERSONAL LOAN_enq_last_24M',
 'num_CONSUMER LOAN_enq_last_9M',
 'total_emi',
 'num_of_loans_le_1lac_non_credit',
 'num_CREDIT CARD_enq_last_3M',
 'Utilization_all',
 'Total_outstanding_Balance_CREDIT CARD_opened_last_6M',
 'Age_of_oldest_open_account',
 'Utilization_CREDIT CARD_opened_last_36M',
 'highest_sanction_amount_non_credit',
 'Total_high_credit',
 'num_PL_LE_1_Lac_opened_in_last_3M',
 'cash_withdrawal_sum',
 'balance_5th_day_M2',
 'bounced_transactions_count',
 'Min_Balance_Min_M1_M2_M3',
 'num_credit_gt_salary',
 'upi_trans_count',
 'Max_Credit_Avg_M1_M2_M3',
 'ratio_total_debit_to_total_credit_M3',
 'ratio_6th_day_balance_to_2th_day_balance_M1',
 'percentage_ever_delinquent',
 'months_since_most_recent_delinquency_all',
 'percentage_utilization_gt_75',
 'No of enq in past 90days',
 'Num_Enq_non_BANK_last_12M',
 'Num_of_installment_loan_opened',
 'g406s',
 'dm206s',
 'statedIncome',
 'deviation_from_mean_income#last_30_Days',
 'deviation_from_median_income/standard_devi

In [63]:
features_list = ['num_PERSONAL LOAN_enq_last_24M',
 'num_CONSUMER LOAN_enq_last_9M',
 'total_emi',
 'num_of_loans_le_1lac_non_credit',
 'num_CREDIT CARD_enq_last_3M',
 'Utilization_all',
 'Total_outstanding_Balance_CREDIT CARD_opened_last_6M',
 'Age_of_oldest_open_account',
 'Utilization_CREDIT CARD_opened_last_36M',
 'highest_sanction_amount_non_credit',
 'Total_high_credit',
 'num_PL_LE_1_Lac_opened_in_last_3M',
 'cash_withdrawal_sum',
 'balance_5th_day_M2',
 'bounced_transactions_count',
 'Min_Balance_Min_M1_M2_M3',
 'num_credit_gt_salary',
 'upi_trans_count',
 'Max_Credit_Avg_M1_M2_M3',
 'ratio_total_debit_to_total_credit_M3',
 'ratio_6th_day_balance_to_2th_day_balance_M1',
 'percentage_ever_delinquent',
 'months_since_most_recent_delinquency_all',
 'percentage_utilization_gt_75',
 'No of enq in past 90days',
 'Num_Enq_non_BANK_last_12M',
 'Num_of_installment_loan_opened',
 'g406s',
 'dm206s',
 'statedIncome',
 'deviation_from_mean_income#last_30_Days',
 'deviation_from_median_income/standard_deviation_income#last_365_Days',
 'yearOfBirth_mean',
'month_year',
'90_in_24mob']

In [64]:
# Selected Features for model build purpose
model_df = model_df1[features_list]
oot_df = oot_df1[features_list]

# Separate features (X) and target (y) for Model and OOT datasets
X_model = model_df.drop(columns=['month_year', '90_in_24mob']) # Adjust 'target' to your actual target column name
y_model = model_df['90_in_24mob']

X_oot = oot_df.drop(columns=['month_year', '90_in_24mob'])
y_oot = oot_df['90_in_24mob']

#Handling Null Values
for i in X_model.columns:
    X_model[i].fillna(-9999,inplace=True)

for i in X_oot.columns:
    X_oot[i].fillna(-9999,inplace=True)
    
X_train, X_test, y_train, y_test = train_test_split(X_model, y_model, test_size=0.40,random_state=42)

# 2. **Prepare DMatrix for XGBoost**
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)
doot = xgb.DMatrix(X_oot, label=y_oot)


# 4. **Predictions**
X_train['y_actual'] = y_train
X_train['15_24_y_pred'] = loaded_model.predict(dtrain)
X_train['LID'] = model_df1.loc[X_train.index,'latest_feature_id'].values

X_test['y_actual'] = y_test
X_test['15_24_y_pred'] = loaded_model.predict(dtest)
X_test['LID'] = model_df1.loc[X_test.index,'latest_feature_id'].values

X_oot['y_actual'] = y_oot
X_oot['15_24_y_pred'] = loaded_model.predict(doot)
X_oot['LID'] = oot_df1.loc[X_oot.index,'latest_feature_id'].values


# 6. **Run Decile Analysis**
train_summary, train_auc, train_gini, train_ks = decile_analysis(X_train, '15_24_y_pred', 'y_actual')
test_summary, test_auc, test_gini, test_ks = decile_analysis(X_test, '15_24_y_pred', 'y_actual')
oot_summary, oot_auc, oot_gini, oot_ks = decile_analysis(X_oot, '15_24_y_pred', 'y_actual')

# 7. **Print Results**
print(f"Train AUC: {train_auc:.2f}, Gini: {train_gini:.2f}, KS: {train_ks:.2f}")
print(f"Test AUC: {test_auc:.2f}, Gini: {test_gini:.2f}, KS: {test_ks:.2f}")
print(f"OOT AUC: {oot_auc:.2f}, Gini: {oot_gini:.2f}, KS: {oot_ks:.2f}")


Train AUC: 0.64, Gini: 0.28, KS: 0.21
Test AUC: 0.65, Gini: 0.30, KS: 0.22
OOT AUC: 0.65, Gini: 0.30, KS: 0.23


/tmp/ipykernel_3180444/736859902.py:14: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  X_model[i].fillna(-9999,inplace=True)
/tmp/ipykernel_3180444/736859902.py:17: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using

In [65]:
Train = X_train[['LID','y_actual','15_24_y_pred']]
Test = X_test[['LID','y_actual','15_24_y_pred']]
Oot = X_oot[['LID','y_actual','15_24_y_pred']]


In [66]:
Train.to_csv('BBILA_Train_Pred_G15_B24_Vin.csv',index=False)
Test.to_csv('BBILA_Test_Pred_G15_B24_Vin.csv',index=False)
Oot.to_csv('BBILA_Oot_Pred_G15_B24_Vin.csv',index=False)


In [67]:
# 60-15
b1_train = pd.read_csv('BBILA_Train_Pred_60_15_Vin.csv')
b1_test = pd.read_csv('BBILA_Test_Pred_60_15_Vin.csv')
b1_oot = pd.read_csv('BBILA_Oot_Pred_60_15_Vin.csv')

#15-24
b1_train_1 = pd.read_csv('BBILA_Train_Pred_G15_B24_Vin.csv')
b1_test_1 = pd.read_csv('BBILA_Test_Pred_G15_B24_Vin.csv')
b1_oot_1 = pd.read_csv('BBILA_Oot_Pred_G15_B24_Vin.csv')

In [68]:
b1_train.drop(columns=['y_actual'],inplace=True)

In [69]:
Train = pd.merge(b1_train,b1_train_1, how='left',on='LID')

In [70]:
b1_test.drop(columns=['y_actual'],inplace=True)
b1_oot.drop(columns=['y_actual'],inplace=True)


In [71]:
Test= pd.merge(b1_test,b1_test_1, how='left',on='LID')


In [72]:
Oot= pd.merge(b1_oot,b1_oot_1, how='left',on='LID')


In [73]:
Train.to_csv('BBILA_Train_Data_ensemble_Vin.csv',index=False)
Test.to_csv('BBILA_Test_Data_ensemble_Vin.csv',index=False)
Oot.to_csv('BBILA_Oot_Data_ensemble_Vin.csv',index=False)
